In [1]:
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [2]:
train_df['label'].value_counts( )

label
0    75173
1    74827
Name: count, dtype: int64

In [3]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# id 칼럼 삭제 수행
train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)


In [4]:
# konlpy: 한국어 자연어 처리를 위한 상위 모듈
# from konlpy.tag import Twitter
from konlpy.tag import Okt

# Okt: 한국어 형태소를 분석하기 위한 형태소 분석기, Twitter의 상위 버전
okt = Okt()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
# min_df:  특정단어가 전체 문서에서 인자값보다 작다면 해당 단어는 무시한다.
# 예) 3이면 특정단어는 최소한 3이상 나와야 분석이 가능하다.
# max_df: 특정단어가 전체 문서의 90%이상 등장하면 해당 단어는 무시한다.
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

In [7]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# LGBM 분류기 초기화
lgbm = LGBMClassifier(random_state=0, n_jobs=-1)

#하이퍼파라메터의 분포를 정의
param_dist = {
    'num_leaves': sp_randint(6, 50), 
    'min_child_samples': sp_randint(100, 500), 
    'min_child_weight': [1e-5, 1e-2, 0.1, 1, 100], 
    'subsample': sp_uniform(loc=0.2, scale=0.8), 
    'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
    'reg_alpha': [0, 1e-1, 1, 2, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 100]
}

# param_dist = {
#     'num_leaves': sp_randint(20, 40),  # 너무 크지 않은 범위로 조정
#     'min_child_samples': sp_randint(50, 150),  # 더 낮은 범위로 조정하여 더 많은 데이터 샘플 고려
#     'min_child_weight': [0.001, 0.01, 0.1, 1],  # 더 세밀한 범위
#     'subsample': sp_uniform(loc=0.7, scale=0.3),  # 더 많은 샘플 사용
#     'colsample_bytree': sp_uniform(loc=0.5, scale=0.5),  # 더 많은 피처 사용
#     'reg_alpha': [0, 0.1, 0.5, 1],  # L1 정규화 조정
#     'reg_lambda': [0, 0.1, 1, 5],  # L2 정규화 조정
#     'max_depth': sp_randint(3, 15)  # 최대 깊이 추가
# }


# RandomizedSearchCV 객체 생성
rnd_search = RandomizedSearchCV(estimator=lgbm, param_distributions=param_dist, 
                                n_iter=100, scoring='accuracy', cv=3, random_state=0, verbose=1)

# 모델 학습
rnd_search.fit(tfidf_matrix_train, train_df['label'])

print('Best parameters found: ', rnd_search.best_params_)
print('Best accuracy found: ', rnd_search.best_score_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Info] Number of positive: 49885, number of negative: 50115
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.086159 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 156038
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 974
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498850 -> initscore=-0.004600
[LightGBM] [Info] Start training from score -0.004600
[LightGBM] [Info] Number of positive: 49885, number of negative: 50115
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.085131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 157338
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 984
[LightGBM] 

In [8]:
# 테스트 데이터 세트에 대한 예측 수행
best_estimator = rnd_search.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

In [9]:
from sklearn.metrics import accuracy_score, f1_score


accuracy = accuracy_score(test_df['label'], preds)
f1 = f1_score(test_df['label'], preds)

print(f'lightGBM 정확도: {accuracy:.4f}')
print(f'lightGBM F1-스코어: {f1:.4f}')

lightGBM 정확도: 0.7961
lightGBM F1-스코어: 0.7901
